In [65]:
from dotenv import load_dotenv
import os
import pandas as pd
import requests
import sys
sys.path.append('/mnt/c/Users/cleon/Documents/CAL_CAL/ironhack/Proyectos/Headquarters-Hunt/src')
import apifunctions as af
from pymongo import MongoClient
import json
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
import shapely.geometry
from pymongo import GEOSPHERE

In [66]:
load_dotenv()

True

In [67]:
client_id = os.getenv("tok1")
client_secret = os.getenv("tok2")

In [68]:
sim_comp = pd.read_csv('../data/similarcompanies.csv')

In [70]:
cities = {
    'Vancouver': [sim_comp['latitude'][0], sim_comp['longitude'][0]],
    'BeverlyH' : [sim_comp['latitude'][1], sim_comp['longitude'][1]],
    'Irvine': [sim_comp['latitude'][2], sim_comp['longitude'][2]]
}

In [71]:
techstartups = '4bf58dd8d48988d125941735'
vegans = '4bf58dd8d48988d1d3941735'
airport = '4bf58dd8d48988d1ed931735'
designstudio = '4bf58dd8d48988d1f4941735'
elemschool = '4f4533804b9074f6e4fb0105'
bar = '4bf58dd8d48988d116941735'
basketstad = '4bf58dd8d48988d18b941735'

venues = [techstartups, vegans, airport, designstudio, elemschool, bar, basketstad]

In [72]:
Vancouver = []
BeverlyH = []
Irvine = []
radius = 2000

for k,v in cities.items():
    if k == 'Vancouver':
        for cat in venues:
            Vancouver.append(af.foursquare(v[0],v[1], cat,radius))
        Vancouver.append(af.foursquarequery(v[0],v[1], 'Starbucks',radius))
    elif k == 'BeverlyH':
        for cat in venues:
            BeverlyH.append(af.foursquare(v[0],v[1], cat,radius))
        BeverlyH.append(af.foursquarequery(v[0],v[1], 'Starbucks',radius))
    elif k == 'Irvine':
        for cat in venues:
            Irvine.append(af.foursquare(v[0],v[1], cat,5000))
        Irvine.append(af.foursquarequery(v[0],v[1], 'Starbucks',radius))

## Now we transform this data to obtain just the parameters we need: Name, location (latitude and longitud) and category

In [73]:
Vancmongo = []

for i in Vancouver:
    Vancmongo.append(af.extraetodo(i))
Vancmongo = [i for lista in Vancmongo for i in lista]

In [74]:
Bevmongo = []

for i in BeverlyH:
    Bevmongo.append(af.extraetodo(i))
Bevmongo = [i for lista in Bevmongo  for i in lista]

In [75]:
Irvmongo = []

for i in Irvine:
    Irvmongo.append(af.extraetodo(i))
Irvmongo = [i for lista in Irvmongo  for i in lista]

## Now, we export the data of each city to a collection in Mongo


In [52]:
client = MongoClient("localhost:27017")
db = client.get_database("Headquarters")


In [76]:
vanc = pd.DataFrame(Vancmongo)
bev = pd.DataFrame(Bevmongo)
irv = pd.DataFrame(Irvmongo)

### Vancouver

In [40]:
gdfvan = gpd.GeoDataFrame(vanc, geometry=gpd.points_from_xy(vanc.longitud, vanc.latitud))

In [42]:
gdfvan.drop('location', axis=1,inplace=True)

In [46]:
gdfvan['geometry']=gdfvan['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdfvan_dic = gdfvan.to_dict(orient="records")


/home/cleonortiz/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [56]:
db.Vancouver.create_index([("geometry", GEOSPHERE)])
db.Vancouver.insert_many(gdfvan_dic)

### Beverly Hills

In [47]:
gdfbev = gpd.GeoDataFrame(bev, geometry=gpd.points_from_xy(bev.longitud, bev.latitud))
gdfbev.drop('location', axis=1,inplace=True)


In [57]:
gdfbev['geometry']=gdfbev['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdfbev_dic = gdfbev.to_dict(orient="records")

/home/cleonortiz/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [62]:
db.BeverlyHills.create_index([("geometry", GEOSPHERE)])
db.BeverlyHills.insert_many(gdfbev_dic)

### Irvine

In [80]:
gdfirv = gpd.GeoDataFrame(irv, geometry=gpd.points_from_xy(irv.longitud, irv.latitud))
gdfirv.drop('location', axis=1,inplace=True)


In [63]:
gdfirv['geometry']=gdfirv['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdfirv_dic = gdfirv.to_dict(orient="records")

/home/cleonortiz/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [64]:
db.Irvine.create_index([("geometry", GEOSPHERE)])
db.Irvine.insert_many(gdfirv_dic)